### IMPORTS

In [ ]:
!pip install --user Node2Vec

In [ ]:
!pip install --user cdlib

In [ ]:
!pip install --user gensim

In [ ]:
!pip install --user leidenalg 

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import re


import networkx as nx
from cdlib import algorithm
from tqdm import tqdm
import time
from networkx.algorithms import community
from community import community_louvain
from operator import itemgetter


from node2vec import Node2Vec
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

#### Graph creation with Nx

In [4]:
df[['Client ID','Page ID','Data']].head(10)

,Client ID,Page ID,Data
1,7.964209e+17,Page 0,20220521
2,3.330376e+18,Page 1,20220527
3,1.065177e+19,Page 2,20220517
4,3.522686e+18,Page 3,20220529
5,3.522686e+18,Page 4,20220529
6,6.440111e+18,Page 5,20220520
7,1.462276e+18,Page 6,20220502
8,9.844695e+18,Page 7,20220512
9,1.656462e+19,Page 8,20220511
10,1.656462e+19,Page 9,20220511


### COLLAPSED

In [ ]:
edges={}
for cl in tqdm(set(df['Client ID'])):
    this_client_df=df[df['Client ID']==cl]
    if len(this_client_df)<=1:
        continue
    for p in this_client_df['Page ID'].tolist():
        for i in range(len(this_client_df['Page ID'])):
            if p==this_client_df['Page ID'].iloc[i]:
                continue
            tupla=(p,this_client_df['Page ID'].iloc[i])
            if tupla in edges:
                edges[tupla] += 1
            else:
                new_tupla=(tupla[1],tupla[0])
                if new_tupla in edges:
                    continue
                else:
                    edges[tupla] = 1

#Filter to keep only the pages that have been visited together more than once
filtered_edges_it={k: v for k, v in edges.items() if v>= 2 }
del edges

#### Graph creation with Nx

In [ ]:
DG_it = nx.DiGraph()
for k, v in filtered_edges_it.items():
    source = k[0]
    target = k[1]
    weight = v
    DG_it.add_edge(source, target, weight = weight)
    
print(nx.info(DG_it))

In [ ]:
# Compute metrics within your network
degree_dict= nx.degree_centrality(DG_it) # Compute the degree centrality for nodes.
nx.set_node_attributes(DG_it, degree_dict, 'degree centrality')

indegree_dict= nx.in_degree_centrality(DG_it) # Compute the in-degree centrality for nodes.
nx.set_node_attributes(DG_it, indegree_dict, 'in-degree centrality')

outdegree_dict= nx.out_degree_centrality(DG_it) # Compute the out-degree centrality for nodes.
nx.set_node_attributes(DG_it, outdegree_dict, 'out-degree centrality')

In [ ]:
G = DG_it.to_undirected()
del DG_it

#### CREATE NODE EMBEDDINGS WITH NODE2VEC

In [ ]:
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=8,p=2,q=0.5)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [ ]:
# Save model for later use
#model.save('Node2Vec_p2_q05.mod')

#### NODE SIMILARITY TOOL

In [ ]:
# Load model
#model = Word2Vec.load("Node2Vec_p2_q05.mod")

In [ ]:
#FIND THE CORRECT NAME OF THE PAGE OF INTEREST WITHIN THE DB
topic='artificial intelligence'

mylist = df['Page ID'].unique()
r = re.compile(f".*{topic}*")

newlist = list(filter(r.match, mylist))
print(newlist)
del mylist

#### MOST SIMILAR PAGES

In [ ]:
#SET A TARGET PAGE
page_name='Page 1'

In [ ]:
model.wv.most_similar(page_name)

#### SIMILARITY BETWEEN USER DEFINED PAGES

In [ ]:
A=model.wv[page_name]

results={}
for page in ['Page 1','Page2','Page 3']:
    v = model.wv[page]
    result=cosine_similarity(A.reshape(1,-1),v.reshape(1,-1))
    results[page]=result[0][0]
{k: v for k, v in sorted(results.items(), key=lambda item: item[1],reverse=True)}